# Data Ingestion

This notebook is the first in the series of notebooks created for the _Nimbus_ demo. Here, we will demonstrate how you can programatically ingest raw data into a database for your data science projects. We will read a set of dataset files from an object store (Amazon S3), convert it into a format compatible with Trino, create a table for this schema, and finally upload the data to this table.

In [1]:
import os
import re
import boto3
import pathlib

import trino

import pandas as pd

from dotenv import dotenv_values, load_dotenv

In [2]:
# Author: Erik Erlandson <eje@redhat.com>
_wsdedup = re.compile(r"\s+")
_usdedup = re.compile(r"__+")
_rmpunc = re.compile(r"[,.()&$/+-]+")
# 63 seems to be a common max column name length
def snakify(name, maxlen=63):
    if isinstance(name, list):
        return [snakify(e) for e in name]
    w = name.casefold().rstrip().lstrip()
    w = w.replace("-", "_")
    w = _rmpunc.sub("", w)
    w = _wsdedup.sub("_", w)
    w = _usdedup.sub("_", w)
    w = w.replace("average", "avg")
    w = w.replace("maximum", "max")
    w = w.replace("minimum", "min")
    w = w.replace("absolute", "abs")
    w = w.replace("source", "src")
    w = w.replace("distribution", "dist")
    # these are common in the sample names but unsure of standard abbv
    #w = w.replace("inference", "inf")
    #w = w.replace("emissions", "emis")
    #w = w.replace("intensity", "int")
    #w = w.replace("reported", "rep")
    #w = w.replace("revenue", "rev")
    w = w[:maxlen] 
    return w

def snakify_columns(df, inplace=False, maxlen=63):
    icols = df.columns.to_list()
    ocols = snakify(icols, maxlen=maxlen)
    scols = set(ocols)
    if (len(set(ocols)) < len(ocols)):
        raise ValueError("remapped column names were not unique!")
    rename_map = dict(list(zip(icols,snakify(icols))))
    return df.rename(columns=rename_map, inplace=inplace)

_p2smap = {
    'string': 'varchar',
    'Float64': 'double',
    'Int64': 'bigint'
}

def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))

# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0],t=e[1]) for e in pz])

In [3]:
# load environment variables
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

In [4]:
# Create an S3 client
s3 = boto3.client(
    service_name="s3",
    endpoint_url=os.environ['S3_ENDPOINT'],
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
)

In [5]:
obj = s3.get_object(
    Bucket=os.environ['S3_BUCKET'],
    Key='urgentem/UrgentemDataSampleEmissionsTargetsDec2020.csv')

# load the raw file from the bucket
dfEmissions = (pd.read_csv(obj['Body'])).convert_dtypes()

# convert columns to specific data types
dfEmissions = dfEmissions.convert_dtypes()

# map column names to a form that works for SQL
snakify_columns(dfEmissions, inplace=True)

# a way to examine the structure of a pandas data frame
dfEmissions.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 15 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   company_name                        19 non-null     string 
 1   isin                                19 non-null     string 
 2   target_type                         19 non-null     string 
 3   scope                               19 non-null     string 
 4   coverage_s1                         16 non-null     Float64
 5   coverage_s2                         15 non-null     Float64
 6   coverage_s3                         4 non-null      Int64  
 7   reduction_ambition                  19 non-null     Float64
 8   base_year                           19 non-null     Int64  
 9   end_year                            19 non-null     Int64  
 10  start_year                          19 non-null     Int64  
 11  base_year_ghg_emissions_s1_tco2e    1 non-null 

In [6]:
dfEmissions.head()

,company_name,isin,target_type,scope,coverage_s1,coverage_s2,coverage_s3,reduction_ambition,base_year,end_year,start_year,base_year_ghg_emissions_s1_tco2e,base_year_ghg_emissions_s1s2_tco2e,base_year_ghg_emissions_s3_tco2e,achieved_reduction
0,3M CO,US88579Y1010,Absolute,S1+S2,1.0,1.0,<NA>,0.5,2002,2025,2015,<NA>,"18,300,000","91,500,000",0.3
1,ADIDAS AG,DE000A1EWWW0,Absolute,S1+S2,0.9,0.9,<NA>,0.15,2015,2020,2015,<NA>,"59,132","295,660",1.0
2,BARCLAYS PLC,GB0031348658,Absolute,S1+S2,1.0,1.0,<NA>,0.37,2018,2025,2018,<NA>,"282,593","1,412,965",0.0
3,DANONE,FR0000120644,Absolute,S1+S2,0.95,0.95,<NA>,0.3,2015,2030,2017,<NA>,"1,681,235","8,406,175",0.68
4,EQUINOR ASA,NO0010096985,Absolute,S1,1.0,<NA>,<NA>,0.21,2016,2030,2017,"9,329,201",<NA>,<NA>,0.06


In [7]:
# parquet has multiple options for appending or updating data
# including adding new files, or appending, sharding directory trees, etc
dfEmissions.to_parquet('/tmp/emissions.parquet', index=False)
s3.upload_file(
    Bucket=os.environ['S3_BUCKET'],
    Key='urgentem/trino/itr_emissions_sample_test/emissions_kachau.parquet',
    Filename='/tmp/emissions.parquet'
)

### FIXME: cant read parquet

In [8]:
# import io

In [9]:
# buffer = io.BytesIO()
# s3_object = s3.Object(os.environ['S3_BUCKET'], 'urgentem/trino/itr_emissions_sample_test/emissions_kachau.parquet')
# s3_object.download_fileobj(buffer)

In [10]:
# obj = s3.get_object(
#     Bucket=os.environ['S3_BUCKET'],
#     Key='urgentem/trino/itr_emissions_sample_test/emissions_kachau.parquet')

# # load the raw file from the bucket
# dfEmissions = (pd.read_parquet(obj['Body'])).convert_dtypes()
# dfEmissions.head()

### END FIXME

In [11]:
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(os.environ['TRINO_USER'], os.environ['TRINO_PASSWD']),
    host=os.environ['TRINO_HOST'],
    port=int(os.environ['TRINO_PORT']),
    http_scheme='https',
    verify=True,
)
cur = conn.cursor()

In [12]:
cur.execute('show columns from default.urgentem.itr_emissions_sample_test')
cur.fetchall()

[['company_name', 'varchar', '', ''],
 ['isin', 'varchar', '', ''],
 ['target_type', 'varchar', '', ''],
 ['scope', 'varchar', '', ''],
 ['coverage_s1', 'double', '', ''],
 ['coverage_s2', 'double', '', ''],
 ['coverage_s3', 'bigint', '', ''],
 ['reduction_ambition', 'double', '', ''],
 ['base_year', 'bigint', '', ''],
 ['end_year', 'bigint', '', ''],
 ['start_year', 'bigint', '', ''],
 ['base_year_ghg_emissions_s1_tco2e', 'varchar', '', ''],
 ['base_year_ghg_emissions_s1s2_tco2e', 'varchar', '', ''],
 ['base_year_ghg_emissions_s3_tco2e', 'varchar', '', ''],
 ['achieved_reduction', 'double', '', '']]

In [13]:
# # For this demonstration example, we just recreate table from scratch each time.
# # in live data platform there will need to be policies and mechanisms for either
# # appending new data, or overwriting old data, or saving off conditioned by a versioning number
# # this is a data governance topic
# cur.execute('drop table if exists hive.urgentem.itr_emissions_sample_test')
# cur.fetchall()

In [14]:
# schema = generate_table_schema_pairs(dfEmissions)

# tabledef = """create table if not exists default.urgentem.itr_emissions_sample_test(
# {schema}
# ) with (
#     format = 'parquet',
#     external_location = 's3a://ocp-odh-os-demo-s3/urgentem/trino/itr_emissions_sample_test/'
# )""".format(schema=schema)
# print(tabledef)

In [15]:
# # tables created externally may not show up immediately in cloud-beaver
# cur.execute(tabledef)
# cur.fetchall()